In [26]:
# Packages need to import
import pandas as pd
import numpy as np
import random
import scipy.stats as stats
from sklearn.metrics.pairwise import cosine_similarity
from statsmodels.stats.weightstats import ttest_ind
import pingouin as pg
from sklearn import preprocessing

In [27]:
# Load data of clothing items ranking
df1 = pd.read_excel('with_prefer_a.xlsx') # including user preferences
df2 = pd.read_excel('without_prefer_a.xlsx') # without including user preferences

In [28]:
# The shopping intents that have to analyse
total_intents = ['Home', 'Business Meeting', 'Gym/Sport', 'Party', 'Outdoor', 'School/Office', 'Meetup with Friends & Family', 'Club or Bar']
total_intents1 = []
for intent in total_intents:
    total_intents1.append(intent.lower())
print(total_intents1)

['home', 'business meeting', 'gym/sport', 'party', 'outdoor', 'school/office', 'meetup with friends & family', 'club or bar']


In [29]:
users1 = df1.shape[0] # number of users along their preferences
users2 = df2.shape[0] # number of user without their preferences

In [30]:
# Function that converts string into list
#ab = '([1,2,3], [2,1,3])'
def convertToList(ab):
    ab = ab[2:-2]
    ab = ab.split('], [')
    a = ab[0]
    b = ab[1]
    c = []
    for each in a:
        if each != ',' and each != ' ':
            c.append(each)
    c= list(map(int, c))
    d = []
    for each in b:
        if each != ',' and each != ' ':
            d.append(each)
    d= list(map(int, d))
    return c, d
#convertToList(ab)

In [31]:
# Get euclidean distance for one intent,.....with the incorporation of shopping preferences
# gym/sport, outdoor, business meeting....#17 for party, #9 for school/office, #2,#9,#17 for home, #9,#67 meetup with friends & family, #17 for club or bar
def per_col_dist1(j):
    col_tau = []
    for i in range(0, users1):
        if (df1.loc[i, j] == 'a'):
            pass
        else:
            ab = df1.loc[i, j]
            a, b = convertToList(ab)
            tau, p_value = stats.kendalltau(np.array([a]), np.array([b]))
            col_tau.append(tau)
    return col_tau
#per_col_dist1('club or bar')

In [32]:
# It calculates distances for all shopping intents with the incorporation of shopping preferences
def all_cols_dist1(total_intents1):
    per_intent_avg_dist = []
    for i in total_intents1:
        x = per_col_dist1(i)
        per_intent_avg_dist.append([i,x])
    return per_intent_avg_dist
#all_cols_dist1(total_intents1)

In [33]:
# Generate a dataframe with all distances for all intents with the incorporation of shopping preferences
score_list1 = all_cols_dist1(total_intents1)
def make_scores_df1(score_list):
    listing = []
    list_header = []
    for each in score_list:
        intent_wise_df = pd.Series(each[1])
        listing.append(intent_wise_df)
        list_header.append(f"{each[0]} with")
    scores_df = pd.concat(listing, axis=1)
    scores_df.columns = list_header
    scores_df = scores_df.assign(id=np.arange(52))
    return scores_df
#make_scores_df1(score_list1)

In [34]:
# Get euclidean distance for one intent,.....without incorporating shopping preferences
# gym/sport, outdoor, business meeting....#17 for party, #9 for school/office, #2,#9,#17 for home, #9,#67 meetup with friends & family, #17 for club or bar
def per_col_dist2(j):
    col_tau = []
    for i in range(0, users2):
        if (df2.loc[i, j] == 'a'):
            pass
        else:
            ab = df2.loc[i, j]
            a, b = convertToList(ab)
            tau, p_value = stats.kendalltau(np.array([a]), np.array([b]))
            col_tau.append(tau)
    return col_tau
#per_col_dist2('club or bar')

In [10]:
# It calculates distances for all shopping intents without incorporating shopping preferences
def all_cols_dist2(total_intents1):
    per_intent_avg_dist = []
    for i in total_intents1:
        x = per_col_dist2(i)
        per_intent_avg_dist.append([i,x])
    return per_intent_avg_dist
#all_cols_dist2(total_intents1)

In [36]:
# Generate a dataframe with all distances for all intents without incorporating shopping preferences
score_list2 = all_cols_dist2(total_intents1)
def make_scores_df2(score_list):
    listing = []
    list_header = []
    for each in score_list:
        intent_wise_df = pd.Series(each[1])
        listing.append(intent_wise_df)
        list_header.append(f"{each[0]} without")
    scores_df = pd.concat(listing, axis=1)
    scores_df.columns = list_header
    scores_df = scores_df.assign(id=np.arange(56))
    return scores_df
#make_scores_df2(score_list2)


In [37]:
# Merge the with and without user preferences based outcomes
df11 = make_scores_df1(score_list1)
df12 = make_scores_df2(score_list2)
final_scores_df1 = df11.merge(df12, how='right', on='id')
final_scores_df1 = final_scores_df1.drop(['id'], axis=1)
final_scores_df1

,home with,business meeting with,gym/sport with,party with,outdoor with,school/office with,meetup with friends & family with,club or bar with,home without,business meeting without,gym/sport without,party without,outdoor without,school/office without,meetup with friends & family without,club or bar without
0,0.066667,0.400000,1.000000,-0.400000,0.466667,0.000000,-0.200000,-0.333333,1.000000,-0.333333,0.200000,1.000000,1.000000,0.400000,0.600000,1.000000
1,1.000000,1.000000,-0.400000,0.400000,0.523810,0.200000,1.000000,1.000000,-0.333333,-0.666667,1.000000,0.000000,0.333333,0.666667,-0.666667,-0.400000
2,1.000000,1.000000,1.000000,1.000000,1.000000,-0.333333,0.333333,1.000000,0.000000,0.333333,-0.333333,-0.600000,-0.333333,-0.333333,1.000000,-1.000000
3,-0.400000,1.000000,1.000000,0.333333,-0.466667,1.000000,1.000000,1.000000,0.666667,-0.333333,0.666667,0.666667,0.733333,0.400000,-0.066667,-0.333333
4,-0.333333,0.600000,1.000000,1.000000,1.000000,-0.333333,0.400000,0.600000,-0.333333,0.666667,1.000000,-0.466667,0.200000,1.000000,-0.333333,0.733333
5,0.000000,0.200000,1.000000,0.333333,0.333333,1.000000,0.714286,1.000000,1.000000,0.000000,-0.400000,0.200000,1.000000,-0.200000,-0.666667,0.000000
6,-0.333333,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,-0.333333,0.666667,1.000000,0.333333,1.000000,0.000000,0.400000,1.000000,-0.466667
7,1.000000,1.000000,0.600000,-1.000000,0.000000,-0.200000,1.000000,0.333333,0.333333,1.000000,-0.400000,0.000000,0.714286,0.200000,0.400000,1.000000
8,0.000000,1.000000,0.000000,1.000000,1.000000,0.066667,0.666667,0.333333,0.200000,-0.333333,0.600000,-0.466667,-1.000000,0.800000,1.000000,1.000000
9,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.600000,0.466667,1.000000,0.800000,-0.333333,0.000000,-0.800000,0.333333,0.666667


In [38]:
# Save the all calculated scores in a file 
final_scores_df1.to_csv('scores_ktau.csv')

In [39]:
# Calculate overall mean and standard devaition for with and without inclusion of user preferences
df11_overall = df11.iloc[:,0:8]
p_mean = np.nanmean(df11_overall) 
p_std = np.nanstd(df11_overall)
print(f" overall mean is {p_mean}")
print(f" overall standard deviation is {p_std}")
df12_overall = df12.iloc[:,0:8]
q_mean = np.nanmean(df12_overall) 
q_std = np.nanstd(df12_overall)
print(f" overall mean is {q_mean}")
print(f" overall standard deviation is {q_std}")

 overall mean is 0.556960978835979
 overall standard deviation is 0.52601407328385
 overall mean is 0.20363558010616833
 overall standard deviation is 0.5740327645855138


In [40]:
# Normalize data
x = final_scores_df1.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
final_scores_df = pd.DataFrame(x_scaled)
final_scores_df.columns= final_scores_df1.columns
print(final_scores_df)

    home with  business meeting with  gym/sport with  party with  \
0    0.440000               0.571429            1.00    0.300000   
1    1.000000               1.000000            0.16    0.700000   
2    1.000000               1.000000            1.00    1.000000   
3    0.160000               1.000000            1.00    0.666667   
4    0.200000               0.714286            1.00    1.000000   
5    0.400000               0.428571            1.00    0.666667   
6    0.200000               1.000000            1.00    1.000000   
7    1.000000               1.000000            0.76    0.000000   
8    0.400000               1.000000            0.40    1.000000   
9    0.400000               0.285714            0.40    1.000000   
10   0.640000               1.000000            1.00    1.000000   
11   0.640000               0.047619            1.00    1.000000   
12   1.000000               0.285714            1.00    0.666667   
13   1.000000               1.000000            

In [41]:
final_scores_df.to_csv('norm_data_ktau.csv')

In [42]:
# Calculate the average and standard deviation values for calculated rank measure
for each in final_scores_df1.columns:
    a = list(final_scores_df1[each].dropna())
    a = np.array(a) 
    print(f"length of {each} is {a.size}")      
    print(f"average of {each} is {a.mean()}")
    print(f"std of {each} is {a.std()}")
   

length of home with is 52
average of home with is 0.47161172161172166
std of home with is 0.5588027943759061
length of business meeting with is 43
average of business meeting with is 0.5864894795127353
std of business meeting with is 0.48118414887819533
length of gym/sport with is 31
average of gym/sport with is 0.610752688172043
std of gym/sport with is 0.5167021907159124
length of party with is 36
average of party with is 0.5064814814814815
std of party with is 0.6355137634105378
length of outdoor with is 49
average of outdoor with is 0.5535471331389699
std of outdoor with is 0.44726795011243903
length of school/office with is 26
average of school/office with is 0.6175824175824176
std of school/office with is 0.5265707594894462
length of meetup with friends & family with is 18
average of meetup with friends & family with is 0.6878306878306879
std of meetup with friends & family with is 0.45097867380656154
length of club or bar with is 33
average of club or bar with is 0.5434343434343

In [43]:
# Calculate an example t-test for shopping intent of "Business Meeting"
for each in final_scores_df1.columns:
    if each=='business meeting with':
        a = list(final_scores_df1[each].dropna())
        a = a[0:51]
    if each=='business meeting without':
        b = list(final_scores_df1[each].dropna())
        b = b[0:51]
result = pg.ttest(a,b,correction=True)
print(result)

               T        dof alternative     p-val        CI95%   cohen-d  \
T-test  2.501544  84.163247   two-sided  0.014302  [0.06, 0.5]  0.535657   

         BF10     power  
T-test  3.303  0.694875  


In [44]:
# It calculates the t-test results for all shopping intents
t_test_result = []
x = final_scores_df1.columns
i = 0; j = 8
for k in range(0, 8):
    a_size = final_scores_df1[x[i]].dropna().values.size
    b_size = final_scores_df1[x[j]].dropna().values.size
    if a_size <= b_size:
        a_sample = random.sample(final_scores_df1[x[i]].dropna().values.tolist(), a_size)
        b_sample = random.sample(final_scores_df1[x[j]].dropna().values.tolist(), a_size)
        a1_size = a_size
        b1_size = a_size
        result = pg.ttest(a_sample,b_sample,correction=True)
        t_test_result.append(a1_size)
        t_test_result.append(result)
    elif a_size > b_size:
        a_sample = random.sample(final_scores_df1[x[i]].dropna().values.tolist(), b_size)
        b_sample = random.sample(final_scores_df1[x[j]].dropna().values.tolist(), b_size)
        a1_size = b_size
        b1_size = b_size
        result = pg.ttest(a_sample,b_sample,correction=True)
        t_test_result.append(a1_size)
        t_test_result.append(result)
    
    i = i + 1
    j = j + 1

t_test_result

[52,
                T         dof alternative     p-val         CI95%   cohen-d  \
 T-test  2.132635  101.040448   two-sided  0.035377  [0.02, 0.43]  0.418244   
 
         BF10     power  
 T-test  1.54  0.560653  ,
 43,
                T        dof alternative     p-val        CI95%   cohen-d  \
 T-test  2.436568  82.532915   two-sided  0.016978  [0.05, 0.5]  0.525484   
 
         BF10     power  
 T-test  2.89  0.673171  ,
 26,
                T       dof alternative     p-val         CI95%   cohen-d  \
 T-test  2.989302  49.34346   two-sided  0.004351  [0.14, 0.72]  0.829083   
 
          BF10     power  
 T-test  9.359  0.834341  ,
 36,
                T        dof alternative     p-val         CI95%  cohen-d  \
 T-test  2.486951  69.137996   two-sided  0.015302  [0.07, 0.65]  0.58618   
 
          BF10     power  
 T-test  3.262  0.688909  ,
 49,
                T        dof alternative     p-val        CI95%   cohen-d  \
 T-test  2.951134  90.100558   two-sided  0.004035  [0

In [45]:
# It makes a text file incorporating all results of t-test calculation
f = open("results_for_t_test_after_sampling_kendall_tau.txt", "a")
x = final_scores_df.columns
i = 0; j = 8
for k in range(0, 8):
    a_size = final_scores_df[x[i]].dropna().values.size
    b_size = final_scores_df[x[j]].dropna().values.size
    if a_size <= b_size:
        a_sample = random.sample(final_scores_df[x[i]].dropna().values.tolist(), a_size)
        b_sample = random.sample(final_scores_df[x[j]].dropna().values.tolist(), a_size)
        a_mean = np.array(a_sample).mean()
        a_std = np.array(a_sample).std()
        b_mean = np.array(b_sample).mean()
        b_std = np.array(b_sample).std()
        a1_size = a_size
        b1_size = a_size
    elif a_size > b_size:
        a_sample = random.sample(final_scores_df[x[i]].dropna().values.tolist(), b_size)
        b_sample = random.sample(final_scores_df[x[j]].dropna().values.tolist(), b_size)
        a_mean = np.array(a_sample).mean()
        a_std = np.array(a_sample).std()
        b_mean = np.array(b_sample).mean()
        b_std = np.array(b_sample).std()
        a1_size = b_size
        b1_size = b_size

    f.write(f'{x[i]}\n {a1_size}, {a_mean}, {a_std}\n')
    f.write(f'{x[j]}\n {b1_size}, {b_mean}, {b_std}\n')
    i = i + 1
    j = j + 1

f.close()


In [46]:
df = pd.read_csv('results_for_t_test_after_sampling_kendall_tau.txt')
print(df.iloc[:,0])

 52                    0.682967    0.335282
home without           NaN              NaN
 52                    0.624359    0.248768
business meeting with  NaN              NaN
 43                    0.684702    0.339004
                                     ...   
 15                    0.531111    0.333881
club or bar with       NaN              NaN
 22                    0.745455    0.294815
club or bar without    NaN              NaN
 22                    0.586364    0.308247
Name: home with, Length: 159, dtype: float64
